In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#def load_data():
global smart
global gas
global weather
df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

In [4]:
smart.head()

,Timestamp,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
0,2017-03-07 07:34:08,3673.459,0.0,4072.481,0.0,89.0,0.0
1,2017-03-07 07:34:18,3673.459,0.0,4072.481,0.0,87.0,0.0
2,2017-03-07 07:34:28,3673.460,0.0,4072.481,0.0,89.0,0.0
3,2017-03-07 07:34:37,3673.460,0.0,4072.481,0.0,89.0,0.0
4,2017-03-07 07:34:47,3673.460,0.0,4072.481,0.0,203.0,0.0


In [5]:
#load_data()

# Change all datetime columns to the same column name
smart = smart.rename(index=str, columns={"Timestamp":"datetime"})
gas = gas.rename(index=str, columns={"gasTimestamp":"datetime"})

#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter', 'T', 'Q']]
#df = df[['datetime', 'eMeter', 'eMeterLow', 'ePower', 'gasMeter']]
smart.head()

# Set datetime as index
smart = smart.set_index(['datetime'])
gas = gas.set_index(['datetime'])

In [6]:
smart.isnull().sum()

eMeter             0
eMeterReturn       0
eMeterLow          0
eMeterLowReturn    0
ePower             0
ePowerReturn       0
dtype: int64

In [7]:
"""
Resample the dataframes to 10s intervals.
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.resample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.interpolate.html#pandas.DataFrame.interpolate
"""
smart = smart.resample('10s').mean()
gas = gas.resample('H').mean() # Does not look nice, but it works

In [8]:
smart.isnull().sum()

eMeter             8566
eMeterReturn       8566
eMeterLow          8566
eMeterLowReturn    8566
ePower             8566
ePowerReturn       8566
dtype: int64

In [9]:
print(smart.memory_usage(index=True).sum()/1000000)

10.861984


In [10]:
gas.head()

,gasMeter
datetime,
2017-03-07 07:00:00,4419.052
2017-03-07 08:00:00,4419.060
2017-03-07 09:00:00,4419.120
2017-03-07 10:00:00,4419.120
2017-03-07 11:00:00,4419.120


# Working with electricity dataframe

In [17]:
#These columns should be created in the end: column_names = ['House_ID','Amount_of_NaNs','Max_consecutive_NaNs', 'Interpolated','Days','File_size','Usable']
#Create a df to store max consecutive NaNs
info_data = pd.DataFrame(columns={'House_ID','File_size','Initial_Date','Final_Date','Days','Amount_of_NaNs','Max_Consecutive_NaNs'})
tempstring = 'P0123423.csv'

def extract_info(df,dwelling_id='test.csv'):

    
    #Process info to get max gap size
    df_nan=df.isnull().any(axis=1)*1
    temp = pd.DataFrame()
    temp['temp']=df_nan.groupby((df_nan != df_nan.shift()).cumsum()).transform('size') * df_nan
    max_gap=temp['temp'].max()
    
    
    #Extract information about dates and number of NaNs
    initial_date_string= df.index[0].strftime('%Y-%m-%d %H:%M:%S')
    final_date_string= df.index[-1].strftime('%Y-%m-%d %H:%M:%S')
    days=df.index[-1]-df.index[0]
    NaN_count=df.isnull().sum()[0]
    
    #Append all information to the global info_data dataframe
    info_data.loc[len(info_data.index)] = [dwelling_id, df.memory_usage(index=True).sum()/1000000, initial_date_string, final_date_string, days, NaN_count, max_gap]
    

extract_info(smart)
print(info_data)

In [ ]:
#Delete rows with NaN in 'ePower' column
smart_final=smart.dropna(subset=['ePower'])
smart_final.isnull().sum()

# Working with gas dataframe

In [ ]:
#Inspect for NaNs
gas_nan=gas.isnull().any(axis=1)
print(gas[gas_nan])

In [ ]:
#Interpolate
gas_final = gas.resample('10s').interpolate(method='time')
# Create a ''gasPower'' column. This equals the gas used in that hour
gas_final['gasPower'] = gas_final['gasMeter'].diff()
# First entry is NaN, replace NaN by second entry
gas_final['gasPower'][0] = gas_final['gasPower'][1]
print(gas_final)

In [ ]:
#Fit datetimes
#gas_selected=gas_final.ix[smart_final.index[0]:gas_final.index[-1]]
#print(gas_selected)

In [ ]:
# Gas datetime ends at 23:00:00 (at least in this specific file) and Electricity date time ends at 23:59:50 so
# when merging final DateTime will be 23:00:00
df = pd.merge(smart_final,gas_final,left_index=True, right_index=True)

In [ ]:
print(df)